IMPORT 

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

GLOBAL VARIABLES

In [92]:
WIN_COUNT = 20000
WIN_SIZE = 5
HID_NODE = 6
NO_CLASS = 6  #INCLUDING CLASS TYPE NONE
CLASS_NAME = ["TIME","PERSON","GEOGRAPHICAL ENTITY","ORGANISATION","OTHERS","GEOPOLITICAL ENTITY"]
READ_COUNT = 0
ONE_TIME = 1000
c4c = 0
weights = []

In [93]:
NO_HID = 1     #NO OF HIDDEN LAYERS

CHOICE FUNCTIONS

In [94]:
def layer_count():
    print("\nPlease choose one of the following:")
    print("\n1. One")
    print("\n2. Three")
    choice = input("\nEnter your choice: ")
    if choice == 1:
        NO_HID = 1
    elif choice == 2:
        NO_HID = 3
    else:
        print("\n\nERROR: Please choose a valid input\n")
        layer_count()

In [95]:
def weight_choice():
    print("\nPlease choose one of the following:")
    print("\n1. Use previously trained weights")
    print("\n2. Use new weights")

    choice = input("\nEnter your choice: ")

    if choice == 1:
    #call a function that uses previous weights
        choice = 1 #RANDOM CODE
    elif choice == 2:
    #call a function that initialises weights as given in assignment
        choice = 2 #RANDOM CODE
    else:
        print("\n\nERROR: Please choose a valid input\n")
        weight_choice()

ACTIVATION FUNCTIONS

In [96]:
def linear(z):
    func = lambda x: x
    line=func(z)
    return line

In [97]:
def relu(z):
    maxi=np.maximum(0,z)
    return maxi

In [98]:
def sigmoid(z):
    part=1+np.exp(-z)
    sigm=1/part
    return sigm

In [99]:
def tanh(z):
    part=1+np.exp(-2*z)
    tan=(2/part)-1
    return tan

In [100]:
def leaky_relu(z, alpha=0.1):
    lrelu=np.maximum(alpha*z,z)
    return lrelu

In [101]:
def softplus(z):
    splus=np.log(1+np.exp(z))
    return splus

In [102]:
def ELU(z, alpha=0.1):
    part=alpha*(np.exp(z)-1)
    eul=np.where(z>=0,z,part)
    return eul

In [103]:
def softmax(z):
    expA = np.exp(z)
    return expA / expA.sum()

CONSTRUCTING NETWORK

In [104]:
def construct_NN(layers_sizes, rand_w):
    H = len(layers_sizes)-1
    global weights
    for h in range(H):
        eh=layers_sizes[h]
        eh1=layers_sizes[h+1]
        if rand_w == 2:
            layer_weights=np.random.uniform(low=0.0, high=0.01,size=eh*eh1).reshape(eh1,eh)
        else:
            layer_weights=np.random.normal(loc=0.005, scale=0.005,size=eh*eh1).reshape(eh1,eh)  
        layer_biases=np.zeros((eh1,1))
        weights.append([layer_weights,layer_biases])
    return weights

FORWARD MODULE

In [105]:
def forward_step(A_prev, W, b, activation, alpha=0.1):
    Z=np.dot(np.array(A_prev).T, np.array(W).T).T+b
    if activation == "relu":
        A = relu(Z)
    elif activation == "sigmoid":
        A = sigmoid(Z)
    elif activation == "tanh":
        A = tanh(Z)
    elif activation == "leaky_relu":
        A = leaky_relu(Z, alpha)
    elif activation == "softplus":
        A = softplus(Z)
    elif activation == "ELU":
        A = ELU(Z, alpha)
    else: # Linear activation
        A = linear(Z)
    return A, Z

In [106]:
def forward_module(X, weights, activations, alpha=0.1):
    layers_cache = []
    H = len(weights)
    A = X
    for h in range(H):
        if h!=0:
            B=A
            A,Y=forward_step(A,weights[h][0],weights[h][1],activations[h],alpha)
            layers_cache.append([B,weights[h][0:1],Y])
        else:
            A,Y=forward_step(X,weights[h][0],weights[h][1],activations[h],alpha)
            layers_cache.append([X,weights[h][0:1],Y])
    
    return A, layers_cache

COST FUNCTIONS

In [107]:
def cross_entropy_cost(AH, Y,reg):
    cost=0
    m = len(AH[0])
    lambd = 10.0
    if reg == 1:
        cost =  -(1.0/m) * (np.dot(np.log(AH), Y) + np.dot(np.log(1-AH), (1-Y)))
    else:
        cost = (np.sum(np.square(W1)) + np.sum(np.square(W2)))*(lambd/(2*m))
    
    return cost

In [108]:
def cross_entropy(prediction, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = prediction.T
    N = predictions.shape[0]
    prob = np.zeros((N,NO_CLASS))
    for i in range(N):
        prob[i] = softmax(predictions[i])
    prob = np.clip(prob, epsilon, 1. - epsilon)
    ce = -np.sum(targets*np.log(prob+1e-9),axis=1)/N
    return ce.sum()



In [109]:
def cross_entropy2(prediction, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    
    predictions = prediction.T
    N = predictions.shape[0]
    ce = np.zeroes(N)
    for i in range(N):
        right = 0
        tw = 0
        for j in range(NO_CLASS):
            sum = sum + prediction[i][j]*targets[i][j]
            tw = tw
        ce[i] = sum
    prob = np.zeros((N,NO_CLASS))
    for i in range(N):
        prob[i] = softmax(predictions[i])
    prob = np.clip(prob, epsilon, 1. - epsilon)
    ce = -np.sum(targets*np.log(prob+1e-9),axis=1)/N
    return ce.sum()

ACTIVATIONS BACKWARD

In [110]:
def linear_backward(dA, Z):
    dZ=dA
    return dZ

In [111]:
def relu_backward(dA, Z):
    dZ=dA*np.where(Z>=0,1,0)
    return dZ

In [112]:
def sigmoid_backward(dA, Z):
    sigm=sigmoid(Z)
    dZ=dA*sigm*(1-sigm)
    return dZ

In [113]:
def tanh_backward(dA, Z):
    tansq=tanh(Z)*tanh(Z)
    dZ=dA*(1-tansq)
    return dZ

In [114]:
def leaky_relu_backward(dA, Z, alpha=0.1):
    dZ=dA*np.where(Z>=0,1,alpha)
    return dZ

In [115]:
def softplus_backward(dA, Z):
    dZ=dA*sigmoid(Z)
    return dZ

In [116]:
def ELU_backward(dA, Z, alpha=0.1):
    part=alpha*np.exp(Z)
    dZ=dA*np.where(Z>=0,1,part)
    return dZ

In [117]:
def softmax_backward(dA, Z):
      dZ=dA*softmax(Z)*(1-softmax(Z))

In [118]:
def activation_backward(dA, Z, activation, alpha=0.1):
    if activation == "relu":
        dZ = relu_backward(dA, Z)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, Z)
    elif activation == "tanh":
        dZ = tanh_backward(dA, Z)
    elif activation == "leaky_relu":
        dZ = leaky_relu_backward(dA, Z, alpha)
    elif activation == "softplus":
        dZ = softplus_backward(dA, Z)
    elif activation == "ELU":
        dZ = ELU_backward(dA, Z, alpha)
    else: # Linear activation
        dZ = linear_backward(dA, Z)   
    return dZ


BACKWARD MODULE

In [119]:
def backward_step(dA, layer_cache, activation, alpha=0.1):
    A_prev = np.array(layer_cache[0])
    W = np.array(layer_cache[1])
    Z = np.array(layer_cache[2])
    dZ=activation_backward(dA,Z,activation,alpha)
    _,m=dA.shape
    dA_prev=np.dot(np.squeeze(W).T,dZ)
    dW=(np.dot(dZ,A_prev.T)) 
    dW = dW/m
    db=np.sum(dZ,axis=1,keepdims=True)/m
    return dA_prev, dW, db

In [120]:
def backward_module(AH, Y, layers_cache, activations, alpha=0.1):
    gradients = []
    H = len(layers_cache) # the number of layers
    m = AH.shape[1]
    Y = Y.T
    dAH=(np.divide(1-Y,1-AH)-np.divide(Y,AH))
    for i in range(H-1,-1,-1):
        dAH,dW,db=backward_step(dAH,layers_cache[i],activations[i],alpha)
        gradients.insert(i,[dW,db])
    return gradients

TRAINING MODULE

In [121]:
def update_weights(weights, gradients, lr):
    H = len(weights)
    for h in range(H):
        weights[h][0]=weights[h][0]-lr*gradients[h][0]
        weights[h][1]=weights[h][1]-lr*gradients[h][1]   
    return weights

In [122]:
def H_layer_NN(X, Y, W, layers_sizes, activations, epochs=200, lr=0.001, alpha=0.1):
    epoch_wise_costs = []  
    weights = W
    global c4c
    pos4 = []
    npx = np.asarray(X)

    for i in range(Y.shape[0]):
        if Y[i][4] == 1 and c4c!=14:
            pos4.append(i)
            c4c = c4c + 1
        elif c4c == 14:
            c4c = 0
        else:
            continue

    X2 = np.delete(X,pos4,1)
    Y2 = np.delete(Y,pos4,0)

    count_class = np.sum(Y2,axis=0)
            
    for epoch in range(epochs):
        AH, layers_cache = forward_module(X2, weights, activations, alpha)
        cost = cross_entropy(AH, Y2)
        gradients = backward_module(AH, Y2, layers_cache, activations, alpha)
        twice = 1       
        update_weights(weights, gradients, lr)
        epoch_wise_costs.append(cost)
        print(".\nCost after epoch: %f" %(cost))

    return weights,cost,count_class

WORD VECTORS READER

In [123]:
embeddings_index = dict() #global variable

f = open("glove.6B.50d.txt",encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


EMBEDDING LAYERS

In [124]:
def read_file(skiprow,nrow):
    with open("ner.csv","r") as f:
        
        data = pd.read_csv (f,skiprows=skiprow,nrows=nrow)   
        df = pd.DataFrame(data, columns= ['prev-prev-word','prev-word','word','next-word','next-next-word','prev-prev-iob','prev-iob','tag','prev-prev-shape','prev-shape','shape','next-shape','next-next-shape'])
    
        return df

In [125]:
def give_class(word):
    if word == 'B-per' or word == 'I-per':
        return 1
    elif word == 'B-geo' or word == 'I-geo':
        return 2
    elif word == 'B-org' or word == 'I-org':
        return 3
    elif word == 'B-gpe' or word == 'I-gpe':
        return 5
    elif word == 'B-tim' or word == 'I-tim':
        return 0
    else:
        return 4

In [126]:
def embedding_layer_output(training_example, window_size, isImprovised, improvedVector, prev_tagged):
    """"
    assuming training example will be a list of 5 words
    if isImprovised is true then the 7 inputs are also passed as a vector. First 5 entries are 0 if lowercase and 1 if upper case for each word. Last two entries are corresponding to tag
    tag of previous two words will have to be made into categorical type: 0,1,2,3,4,5... corresponding to each tag
     training eg should be of the form: [prev-prev, prev, word, next, next-next]
    assuming window size will be either 0,1, or 2
    """
    output = np.zeros(257)
    if(isImprovised):
        output[252:257] = improvedVector
        output[250:252] = prev_tagged
    temp = embeddings_index.get(training_example[2].lower())
    if (training_example[2] in embeddings_index.keys()):
        output[100:150] = temp
    if(window_size>0):
        temp = embeddings_index.get(training_example[1].lower())
        if (training_example[1] in embeddings_index.keys()):
            output[50:100] = temp
        temp = embeddings_index.get(training_example[3].lower())
        if (training_example[3] in embeddings_index.keys()):
             output[150:200] = temp
        if(window_size>1):
            temp = embeddings_index.get(training_example[0].lower())
            if (training_example[0] in embeddings_index.keys()):
                output[50:100] = temp
            temp = embeddings_index.get(training_example[4].lower())
            if (training_example[4] in embeddings_index.keys()):
                output[150:200] = temp
    return output

In [127]:
def initialise(df,sub):
    window_matrix = []
    for ind in df.index: 
        window_matrix.append([df['prev-prev-word'][ind], df['prev-word'][ind], df['word'][ind], df['next-word'][ind], df['next-next-word'][ind]])
  
    shape_matrix = []
    prev_tag = []
    zero = [0,0,0,0,0]
    single = zero
    for ind in df.index:
        single = zero
        if df['prev-prev-shape'][ind] == 'capitalized':
            single[0] = 1
        if df['prev-shape'][ind] == 'capitalized':
            single[1] = 1
        if df['shape'][ind] == 'capitalized':
            single[2] = 1
        if df['next-shape'][ind] == 'capitalized':
            single[3] = 1
        if df['next-next-shape'][ind] == 'capitalized':
            single[4] = 1
        prev_tag.append([give_class(df['prev-prev-iob'][ind]), give_class(df['prev-iob'][ind])])
        shape_matrix.append(single)

    input_matrix = []
    for ind in df.index:
        ind = ind - sub
        temp = embedding_layer_output(window_matrix[ind],2,True,shape_matrix[ind],prev_tag[ind])
        input_matrix.append(temp)
    one_hot_labels = np.zeros((len(df.index), NO_CLASS))
    class_code = 0
    for ind in df.index:
        class_code = give_class(df['tag'][ind])
        ind = ind - sub
        one_hot_labels[ind][class_code] = 1

    return input_matrix, one_hot_labels
  


TRAINING

In [128]:
def train_cont(layers_sizes,activations,reg,rand_w,wind,epochs):
    
    print("LAYERS SIZES",layers_sizes)
    lr = 0.03
    alpha = 0.1
    TOTAL_READ_COUNT = 1000
    READ_COUNT = 0
    ONE_TIME = 1000
    chunk = 1000
    layers_sizes = [257, 50, 6] #Last one is the number of output classes  &&  First one is the number of input nodes
    activations = ["ELU","sigmoid"]
    data = pd.read_csv('ner.csv', encoding='unicode-escape',chunksize = chunk)
    global weights 
    weights = construct_NN(layers_sizes, rand_w)
    ind = 1
    cost_arr = []
    total_count = np.zeros(NO_CLASS)
    for i in range(epochs):
        for  c in data:
            if (ind == 69 or ind == 690):
                ind = ind+1
                continue
            if (ind==10):
                test_set = c
                ind = ind + 1
                continue
            if (ind!=10 and ind%10==0):
                test_set.append(c,ignore_index = True)
                ind = ind+1
                continue      
            ind = ind +1
            if (ind>100):
                break
            input_matrix, one_hot_labels = initialise(c,(ind-2)*chunk)
            input_transpose = [[row[i] for row in input_matrix] for i in range(len(input_matrix[0]))]
            weights,cost,count_class = H_layer_NN(input_transpose, one_hot_labels, weights, layers_sizes, activations, 5, lr, alpha)
            cost_arr.append(cost)
            total_count = total_count + count_class

    print("TOTAL COUNT: ",total_count)
    return weights

TESTING

In [129]:
def predict(X,Y,weights,activations,alpha=0.1,total=10500):
    confuse_matrix = np.zeros((NO_CLASS,NO_CLASS))
    confuse_matrix = np.squeeze(confuse_matrix)
    output, _ = forward_module(X, weights, activations, alpha)
    output = np.squeeze(output)
    output = output.T
    pred_class = np.zeros((total,NO_CLASS))
    max_val = np.amax(output, axis=1)
    actual = 0
    predicted = 0

    for i in range(total):
        predicted = np.asarray(np.where(output[i] == max_val[i]))
        pred_class[i][predicted] = 1
        actual = np.asarray(np.where(Y[i] == 1))
        pred_val = predicted[0][0]
        act_val = actual[0][0]
        confuse_matrix[act_val][pred_val] = confuse_matrix[act_val][pred_val] + 1
  
    accurate_pred = 0
    for i in range(NO_CLASS):
        accurate_pred = accurate_pred + confuse_matrix[i][i]
    accuracy = accurate_pred/total
    print("\nAccuracy:", accuracy)

    precision = np.zeros(NO_CLASS)
    class_total = np.sum(confuse_matrix, axis=0)
    for i in range(NO_CLASS):
        precision[i] = confuse_matrix[i][i]/(class_total[i]+1)
        print("\nF1 Score for class %d[%s] is: %1.3f" %(i+1,CLASS_NAME[i],precision[i]))

In [130]:
def initialise_test(df,sub):
    window_matrix = []
    print(df.index)
    for ind in df.index: 
        window_matrix.append([df['prev-prev-word'][ind], df['prev-word'][ind], df['word'][ind], df['next-word'][ind], df['next-next-word'][ind]])
  
    shape_matrix = []
    prev_tag = []
    zero = [0,0,0,0,0]
    single = zero
    for ind in df.index:
        single = zero
        if df['prev-prev-shape'][ind] == 'capitalized':
            single[0] = 1
        if df['prev-shape'][ind] == 'capitalized':
            single[1] = 1
        if df['shape'][ind] == 'capitalized':
            single[2] = 1
        if df['next-shape'][ind] == 'capitalized':
            single[3] = 1
        if df['next-next-shape'][ind] == 'capitalized':
            single[4] = 1
        prev_tag.append([give_class(df['prev-prev-iob'][ind]), give_class(df['prev-iob'][ind])])
        shape_matrix.append(single)

    input_matrix = []
    for ind in df.index:
        ind = ind - sub
        temp = embedding_layer_output(window_matrix[ind],2,True,shape_matrix[ind],prev_tag[ind])
        input_matrix.append(temp)
    one_hot_labels = np.zeros((len(df.index), NO_CLASS))
    class_code = 0
    for ind in df.index:
        class_code = give_class(df['tag'][ind])
        ind = ind - sub
        one_hot_labels[ind][class_code] = 1

    return input_matrix, one_hot_labels

In [131]:
def give_test():
    for i in range(10):
        sf = read_file(i*ONE_TIME*10,ONE_TIME*10)
        if i == 0:
            tf = sf
        else:
            tf.append(sf, ignore_index=True)
    return tf

In [132]:
def test_cont(wind,activations):
    global weights
    test_set = give_test()
    total = len(test_set.index)
    print("TOTAL",total)
    input_test, output_test = initialise_test(test_set,0)
    test_transpose = [[row[i] for row in input_test] for i in range(len(input_test[0]))]
    np_test = np.asarray(test_transpose)
    if wind == 2:
        np_test[:][0] = 0
        np_test[:][1] = 0
        np_test[:][3] = 0
        np_test[:][4] = 0
    elif wind == 1:
        np_test[:][0] = 0
        np_test[:][4] = 0
    else:
        xyz = 0  ####RANDOM    can also use PASS
    test_transpose = np_test.tolist()
    output_test = np.asarray(output_test)
    print(output_test.shape)
    print("LENGTH",len(test_transpose))
    count_class = np.sum(output_test,axis=0)
    print("COUNT CLASS",count_class)
    predict(test_transpose, output_test, weights, activations, 0.1, 1000)

MAIN MODULE & CO-FUNCTIONS

In [133]:
def train():
    global weights
    weights = []
    
    hidd_nos = input("Choose the number of hidden layers \nMin = 1 and Max = 3\n")
    hid_no = int(hidd_nos)
    layers_sizes = [257]
    if hid_no == 1:
        layers_sizes.append(50)
    elif hid_no == 2:
        layers_sizes.append(50)
        layers_sizes.append(50)
    elif hid_no == 3:
        layers_sizes.append(50)
        layers_sizes.append(50)
        layers_sizes.append(50)
    else:
        print("WRONG CHOICE OF LAYERS SIZE")
    layers_sizes.append(NO_CLASS)
    
    print("Choose the activation functions for the hidden layers")
    print("1.ReLU \n2.Sigmoid \n3.TanH \n4.Leaky ReLU \n5.Softplus \n6.ELU \n7.Linear")
    activations = []
    for i in range(hid_no + 1):
        print("Activation function for layer %d:",i+1)
        take = input("Your Choice: ")
        if take == "1":
            activations.append("relu")
        elif take == "2":
            activations.append("sigmoid")
        elif take == "3":
            activations.append("tanh")
        elif take == "4":
            activations.append("leaky_relu")
        elif take == "5":
            activations.append("softplus")
        elif take == "6":
            activations.append("ELU")
        else:
            activations.append("linear")
    
    reg_s = input("\nDo you want regularization of cost function? \nPress 1 for Yes \nPress 2 for No \n")
    reg = int(reg_s)
    
    rand_s = input("\nHow do you want to initialise weights? \n1.Random \n2.Uniform \n")
    rand_w = int(rand_s)
    
    print("\nYou have the window size choices of 0/1/2")
    wind_s = input("Choose your window size: ")
    wind = int(wind_s)
    wind_l = [wind]
    
    print("\nYou have the epoch choices between 0-100")
    epoch_s = input("Your Choice: ")
    epoch = int(epoch_s)
    
    weights = train_cont(layers_sizes,activations,reg,rand_w,wind,epoch)
    
    start = [wind_l,layers_sizes,activations]
    
    # opening the csv file in 'w+' mode 
    file = open('weights.csv', 'w+', newline ='') 

    # writing the data into the file 
    with file:     
        write = csv.writer(file) 
        write.writerows(start)
        write.writerows(weights)

In [134]:
def test():
    global weights
    weights = [[0,0],[0,0]]
    ini_list = []
    # opening the CSV file 
    with open('weights.csv', mode ='r')as file: 
        # reading the CSV file 
        csvFile = csv.reader(file)
        
        i = 0
        # displaying the contents of the CSV file 
        for lines in csvFile: 
            if i == 1:
                layers_sizes = lines
                i=i+1
            elif i == 2:
                activations = lines
                i=i+1
            elif i == 0:
                wind = lines[0]
                i=i+1
            else:
                ini_list.append(lines)
        for i in range(2):
            for j in range(2):
                str1 = ini_list[i][j]
                weights[i][j] = str1.strip('][').split(', ')
        
        test_cont(wind,activations)
       

In [135]:
print("WELCOME TO THE NER CODE IMPLEMENTATION")
print("The dataset used in the following is 'Annotated Corpus for NER, by Abhinav Walia [KAGGLE]'")
print("\nDo you want to train or test?")
tort = input("Enter 1 for training and 2 for testing: ")

global weights
weights = []
if tort == "1":
    train()
elif tort == "2":
    test()
else:
    print("Wrong choice")

WELCOME TO THE NER CODE IMPLEMENTATION
The dataset used in the following is 'Annotated Corpus for NER, by Abhinav Walia [KAGGLE]'

Do you want to train or test?
Enter 1 for training and 2 for testing: 1
Choose the number of hidden layers 
Min = 1 and Max = 3
2
Choose the activation functions for the hidden layers
1.ReLU 
2.Sigmoid 
3.TanH 
4.Leaky ReLU 
5.Softplus 
6.ELU 
7.Linear
Activation function for layer %d: 1
Your Choice: 1
Activation function for layer %d: 2
Your Choice: 2
Activation function for layer %d: 3
Your Choice: 3

Do you want regularization of cost function? 
Press 1 for Yes 
Press 2 for No 
1

How do you want to initialise weights? 
1.Random 
2.Uniform 
1

You have the window size choices of 0/1/2
Choose your window size: 2

You have the epoch choices between 0-100
Your Choice: 1
LAYERS SIZES [257, 50, 50, 6]
.
Cost after epoch: 1.791624
.
Cost after epoch: 1.791380
.
Cost after epoch: 1.791141
.
Cost after epoch: 1.790903
.
Cost after epoch: 1.790661
.
Cost after ep

.
Cost after epoch: 1.658071
.
Cost after epoch: 1.648816
.
Cost after epoch: 1.642319
.
Cost after epoch: 1.637457
.
Cost after epoch: 1.633615
.
Cost after epoch: 1.665179
.
Cost after epoch: 1.666078
.
Cost after epoch: 1.666652
.
Cost after epoch: 1.666745
.
Cost after epoch: 1.666446
.
Cost after epoch: 1.678947
.
Cost after epoch: 1.677933
.
Cost after epoch: 1.677028
.
Cost after epoch: 1.676213
.
Cost after epoch: 1.675487
.
Cost after epoch: 1.662016
.
Cost after epoch: 1.658539
.
Cost after epoch: 1.655435
.
Cost after epoch: 1.652638
.
Cost after epoch: 1.650096
.
Cost after epoch: 1.667581
.
Cost after epoch: 1.666988
.
Cost after epoch: 1.666510
.
Cost after epoch: 1.666092
.
Cost after epoch: 1.665723
.
Cost after epoch: 1.654006
.
Cost after epoch: 1.650582
.
Cost after epoch: 1.648057
.
Cost after epoch: 1.646063
.
Cost after epoch: 1.644417
.
Cost after epoch: 1.650896
.
Cost after epoch: 1.651521
.
Cost after epoch: 1.652017
.
Cost after epoch: 1.652283
.
Cost after e